<a href="https://colab.research.google.com/github/mitul01/FER/blob/main/FER_manual_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/MyDrive/colab-20210303T131708Z-001/colab

/content/drive/MyDrive/colab-20210303T131708Z-001/colab


In [ ]:
from config import emotion_config as config
from pipeline.preprocessing import ImageToArrayPreprocessor
from pipeline.callbacks import EpochCheckpoint
from pipeline.callbacks import TrainingMonitor
from pipeline.io import HDF5DatasetGenerator
from pipeline.nn.conv import EmotionVGGNet
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.models import load_model
from keras import backend as K
import argparse
import os

from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout,Activation
from tensorflow.keras import Sequential

In [ ]:
trainAug = ImageDataGenerator(rotation_range = 10, zoom_range = 0.1,
    horizontal_flip = True, rescale = 1 / 255.0, fill_mode = "nearest")

valAug = ImageDataGenerator(rescale = 1 / 255.0)
iap = ImageToArrayPreprocessor()

# initialize the training and validation dataset generators

trainGen = HDF5DatasetGenerator(config.TRAIN_HDF5, config.BATCH_SIZE,
    aug = trainAug, preprocessors = [iap], classes = config.NUM_CLASSES)
valGen = HDF5DatasetGenerator(config.VAL_HDF5, config.BATCH_SIZE,
    aug = valAug, preprocessors = [iap], classes = config.NUM_CLASSES)

/content/drive/MyDrive/colab-20210303T131708Z-001/colab/pipeline/io/hdf5datasetgenerator.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.db = h5py.File(dbPath)


In [ ]:
model = Sequential()
model.add(Conv2D(64, (1, 1), padding='same', activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add((Conv2D(128, (3, 3),padding='same', activation='relu')))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv2D(256, (5, 5),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((3, 3),padding="same"))
model.add(Dropout(0.5))


model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(6, activation='softmax'))
model.summary()


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# model.fit(X_train, Y_train, batch_size=64, epochs=40, steps_per_epoch=(len(X_train)/128))


#model.fit(X_train, Y_train, batch_size=64, epochs=40, steps_per_epoch=len(X_train)/128, validation_split = 0.25)
batch_size = 64

epochs = 150
from tensorflow.keras import optimizers
optim=optimizers.Adam(0.001)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=7,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

history=model.fit_generator(
    trainGen.generator(),
    steps_per_epoch = trainGen.numImages // batch_size,
    validation_data = valGen.generator(),
    validation_steps = valGen.numImages // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    verbose = 1
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        128       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 128)       512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 256)       8

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


448/448 [==============================] - 114s 171ms/step - loss: 1.9526 - accuracy: 0.2439 - val_loss: 1.5535 - val_accuracy: 0.3823
Epoch 2/150
448/448 [==============================] - 77s 172ms/step - loss: 1.5361 - accuracy: 0.3782 - val_loss: 1.3281 - val_accuracy: 0.4739
Epoch 3/150
448/448 [==============================] - 77s 171ms/step - loss: 1.3926 - accuracy: 0.4457 - val_loss: 1.2549 - val_accuracy: 0.5042
Epoch 4/150
448/448 [==============================] - 76s 170ms/step - loss: 1.3314 - accuracy: 0.4735 - val_loss: 1.2013 - val_accuracy: 0.5302
Epoch 5/150
448/448 [==============================] - 76s 170ms/step - loss: 1.2860 - accuracy: 0.4963 - val_loss: 1.1692 - val_accuracy: 0.5377
Epoch 6/150
448/448 [==============================] - 76s 170ms/step - loss: 1.2559 - accuracy: 0.5134 - val_loss: 1.1330 - val_accuracy: 0.5533
Epoch 7/150
448/448 [==============================] - 76s 170ms/step - loss: 1.2207 - accuracy: 0.5249 - val_loss: 1.1196 - val_accura